In [1]:
import tensorflow as tf
from tf.keras.layers import Dense, Flatten, Conv2D, Input, Lambda

In [ ]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3 # Cambiar en función de la cantidad de satélites

#### MODEL

In [ ]:
inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255)(inputs)
c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
